In [2]:
import os


def getFilePaths(directory):
    filepaths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg"):
                filepaths.append(os.path.join(root, file))
    return filepaths


def getLabels(filepaths):
    labels = []
    for file in filepaths:
        labels.append(file.split('/')[2])
    return labels

In [3]:
import pandas as pd

train_files = getFilePaths('data/train')
train_labels = getLabels(train_files)
train_df = pd.DataFrame({'filepath': train_files, 'label': train_labels})
train_df.head()

,filepath,label
0,data/train/Paper/image_811.jpg,Paper
1,data/train/Paper/image_805.jpg,Paper
2,data/train/Paper/image_193.jpg,Paper
3,data/train/Paper/image_187.jpg,Paper
4,data/train/Paper/image_839.jpg,Paper


In [4]:
test_df = pd.DataFrame({'filepath': getFilePaths('data/test'), 'label':
    ['test' for i in range(len(getFilePaths('data/test')))]})
test_df.head()

,filepath,label
0,data/test/test_image_1159.jpg,test
1,data/test/test_image_588.jpg,test
2,data/test/test_image_1165.jpg,test
3,data/test/test_image_1171.jpg,test
4,data/test/test_image_239.jpg,test


In [5]:
import tensorflow as tf

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    validation_split=0.2,
    #zoom_range=0.2,
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255
)

In [6]:
img_size = 100
batch_size = 32
epochs = 100

In [7]:
import keras
from keras import layers

def build_model():
    model = keras.Sequential()
    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
import numpy as np
batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    subset="training",
    batch_size=batch_size,
    target_size=(img_size, img_size),
    shuffle=True,
    seed=1234,
    color_mode='grayscale'
)

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    subset="validation",
    batch_size=batch_size,
    target_size=(img_size, img_size),
    shuffle=True,
    seed=1234,
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    batch_size=1,
    target_size=(img_size, img_size),
    shuffle=False,
    color_mode='grayscale'
)

model = build_model()

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

print("Average train accuracy: ", np.mean(train_acc))
print("Average train loss: ", np.mean(train_loss))
print("Average val accuracy: ", np.mean(val_acc))
print("Average val loss: ", np.mean(val_loss))

y_pred = model.predict(test_generator)

y_pred_classes = np.argmax(y_pred, axis=1)
class_indices_list = list(train_generator.class_indices.keys())
print(class_indices_list)
y_pred_classes = [class_indices_list[i] for i in y_pred_classes]

Found 2520 validated image filenames belonging to 3 classes.
Found 630 validated image filenames belonging to 3 classes.
Found 1350 validated image filenames belonging to 1 classes.
Epoch 1/100


2023-12-14 00:06:40.822169: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


79/79 [==============================] - 3s 31ms/step - loss: 0.7163 - accuracy: 0.6762 - val_loss: 0.8048 - val_accuracy: 0.6825
Epoch 2/100
79/79 [==============================] - 3s 32ms/step - loss: 0.3343 - accuracy: 0.8817 - val_loss: 0.1382 - val_accuracy: 0.9587
Epoch 3/100
79/79 [==============================] - 2s 30ms/step - loss: 0.2017 - accuracy: 0.9361 - val_loss: 0.4421 - val_accuracy: 0.8222
Epoch 4/100
79/79 [==============================] - 3s 32ms/step - loss: 0.1519 - accuracy: 0.9488 - val_loss: 0.5179 - val_accuracy: 0.8032
Epoch 5/100
79/79 [==============================] - 3s 32ms/step - loss: 0.0770 - accuracy: 0.9750 - val_loss: 0.1644 - val_accuracy: 0.9508
Epoch 6/100
79/79 [==============================] - 3s 33ms/step - loss: 0.0464 - accuracy: 0.9881 - val_loss: 0.1595 - val_accuracy: 0.9444
Epoch 7/100
79/79 [==============================] - 3s 32ms/step - loss: 0.0263 - accuracy: 0.9948 - val_loss: 0.2810 - val_accuracy: 0.9095
Epoch 8/100
79/79 

In [9]:
data = {'image': test_df['filepath'].apply(lambda x: os.path.basename(x)), 'predicted_class': y_pred_classes}
df = pd.DataFrame(data=data)
df.head()

,image,predicted_class
0,test_image_1159.jpg,Rock
1,test_image_588.jpg,Rock
2,test_image_1165.jpg,Paper
3,test_image_1171.jpg,Scissors
4,test_image_239.jpg,Paper


In [10]:
model.save('classifier.h5')